In [2]:
import pandas as pd
# Antar at datasettet heter tonsberg.xlsx, med tre ark: helg, rt og ur
helg = pd.read_excel('tonsberg.xlsx', sheet_name='HELG')

rush = pd.read_excel('tonsberg.xlsx', sheet_name='RT')

kveld = pd.read_excel('tonsberg.xlsx', sheet_name='UR')

In [3]:


nye_navn = {

    'fra-sone': 'fra_sone',
    'fil-sone': 'til_sone',
    'fra-Grk': 'fra_grk',
    'Til-Grk': 'til_grk',
    'Reiser': 'reiser',
    'Ndays': 'dager',
    'Arb': 'arbeidsplasser',
    'Distance': 'distanse',
    'Handel': 'handel',
    'FriHenPriv': 'fri_hen_priv',
    'TotBef': 'befolkning'
}


helg = helg.rename(columns=nye_navn)

rush = rush.rename(columns=nye_navn)

kveld = kveld.rename(columns=nye_navn)

In [22]:
data_helg = helg.drop(['Gange', 'Sykkel', 'Bil', 'Bilpass', 'PT', 'dager'], axis=1)
data_rush = rush.drop(['Gange', 'Sykkel', 'Bil', 'Bilpass', 'PT', 'dager'], axis=1)
data_kveld = kveld.drop([ 'Gange', 'Sykkel', 'Bil', 'Bilpass', 'PT', 'dager'], axis=1)

In [24]:
data_kveld['fra-sone-6'] = data_kveld['fra_grk'].astype(str).str[:7]  
data_kveld['til-sone-6'] = data_kveld['til_grk'].astype(str).str[:7]  

data_rush['fra-sone-6'] = data_rush['fra_grk'].astype(str).str[:7]  
data_rush['til-sone-6'] = data_rush['til_grk'].astype(str).str[:7]  

data_helg['fra-sone-6'] = data_helg['fra_grk'].astype(str).str[:7]  
data_helg['til-sone-6'] = data_helg['til_grk'].astype(str).str[:7]  

In [25]:
# Steg 1: Aggregér alle variabler unntatt `TotBef`
data_grouped_kveld = data_kveld.groupby(['fra-sone-6', 'til-sone-6'], as_index=False).agg({
    'reiser': 'sum',
    'arbeidsplasser': 'sum',
    'distanse': 'mean',
    'handel': 'sum',
    'fri_hen_priv': 'sum'
})

# Steg 2: Finn unike `TotBef` per `fra-sone-6`
totbef_per_frasone = data_kveld.groupby('fra-sone-6')['befolkning'].unique().apply(sum).reset_index()

# Steg 3: Slå sammen `TotBef` tilbake til `data_grouped`, basert på `fra-sone-6`
data_grouped_kveld = data_grouped_kveld.merge(totbef_per_frasone, on='fra-sone-6', how='left')

data_grouped_rush = data_rush.groupby(['fra-sone-6', 'til-sone-6'], as_index=False).agg({
    'reiser': 'sum',
    'arbeidsplasser': 'sum',
    'distanse': 'mean',
    'handel': 'sum',
    'fri_hen_priv': 'sum'
})

totbef_per_frasone_rush = data_rush.groupby('fra-sone-6')['befolkning'].unique().apply(sum).reset_index()

data_grouped_rush = data_grouped_rush.merge(totbef_per_frasone_rush, on='fra-sone-6', how='left')

data_grouped_helg = data_helg.groupby(['fra-sone-6', 'til-sone-6'], as_index=False).agg({
    'reiser': 'sum',
    'arbeidsplasser': 'sum',
    'distanse': 'mean',
    'handel': 'sum',
    'fri_hen_priv': 'sum'
})

totbef_per_frasone_helg = data_helg.groupby('fra-sone-6')['befolkning'].unique().apply(sum).reset_index()

data_grouped_helg = data_grouped_helg.merge(totbef_per_frasone_helg, on='fra-sone-6', how='left')



In [26]:
# prompt: Legg til en kolonne i alle tre groped-datasett som heter tid, og skriv henholdvis "UR", "RT" og "HELG"

# Add a 'tid' column to each DataFrame
data_grouped_kveld['tid'] = 'UR'
data_grouped_rush['tid'] = 'RT'
data_grouped_helg['tid'] = 'HELG'

In [27]:
# slå sammen data_grouped til ett datasett
data_grouped = pd.concat([data_grouped_kveld, data_grouped_rush, data_grouped_helg], ignore_index=True)

In [28]:
data_grouped.head()

,fra-sone-6,til-sone-6,reiser,arbeidsplasser,distanse,handel,fri_hen_priv,befolkning,tid
0,6040601,6040601,14074.3777,2350,48.13,976,1063,937.66,UR
1,6040601,7010101,334.7720,64,70.48,1,40,937.66,UR
2,6040601,7010102,332.6725,23,67.50,5,14,937.66,UR
3,6040601,7010103,443.7407,48,65.09,0,43,937.66,UR
4,6040601,7010104,1161.4044,134,64.43,34,87,937.66,UR


In [30]:

# Save the DataFrame to a CSV file in your Google Drive
data_grouped.to_csv('/content/drive/MyDrive/Master/data_grouped.csv', index=False)
